## Neural Networks

Load libraries

In [1]:
import torch
import random
import numpy as np
from sklearn import datasets
from sklearn import model_selection
from typing import Type

Set up session for reproducibility

In [2]:
# Python
random.seed(0)
# Numpy
np.random.seed(1)
# PyTorch
torch.manual_seed(2)
# Ensure that all operations are deterministic on GPU (if used)
torch.backends.cudnn.determinstic = True
torch.backends.cudnn.benchmark = False
# Avoid nondeterministic algorithms
torch.use_deterministic_algorithms(True)

Load the [Iris dataset](https://archive.ics.uci.edu/dataset/53/iris).

In [3]:
dataset = datasets.load_iris()
x = dataset.data
y = dataset.target

Describe the data

In [4]:
print(f"x shape: {x.shape}")
print(f"  - Feature 1: sepal length (cm)")
print(f"  - Feature 2: sepal width (cm)")
print(f"  - Feature 3: petal length (cm)")
print(f"  - Feature 4: petal width (cm)")
print(f"--------")
print(f"y shape: {y.shape}")
print(f"  - Label 1: class (three classes)")

x shape: (150, 4)
  - Feature 1: sepal length (cm)
  - Feature 2: sepal width (cm)
  - Feature 3: petal length (cm)
  - Feature 4: petal width (cm)
--------
y shape: (150,)
  - Label 1: class (three classes)


Split the data intro two groups: train and test

In [5]:
x_train, x_test, y_train, y_test = model_selection.train_test_split(x, y, test_size=0.2, random_state=1)
print(f"x_train shape: {x_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"--------")
print(f"x_test shape: {x_test.shape}")
print(f"y_test shape: {y_test.shape}")

x_train shape: (120, 4)
y_train shape: (120,)
--------
x_test shape: (30, 4)
y_test shape: (30,)


Transform arrays to tensors

In [6]:
x_train_tensor = torch.tensor(x_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.int64)

x_test_tensor = torch.tensor(x_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.int64)

Build PyTorch datasets and dataloaders

In [7]:
dataset_train = torch.utils.data.TensorDataset(x_train_tensor, y_train_tensor)
dataset_test  = torch.utils.data.TensorDataset(x_test_tensor,  y_test_tensor)

batch_size = 5
dataloader_train = torch.utils.data.DataLoader(dataset=dataset_train, batch_size=batch_size, shuffle=True)
dataloader_test  = torch.utils.data.DataLoader(dataset=dataset_test,  batch_size=batch_size, shuffle=True)

Define the neural network architecture

In [8]:
class NeuralNetwork(torch.nn.Module):
    def __init__(self, input_size: int, hidden_size: int, output_size: int) -> None:
        super().__init__()
        self.layer1 = torch.nn.Linear(input_size,  hidden_size)
        self.layer2 = torch.nn.Linear(hidden_size, output_size)
        self.activation = torch.nn.ReLU()

    def forward(self, x) -> torch.Tensor:
        x = self.layer1(x)
        x = self.activation(x)
        x = self.layer2(x)
        return x

Build a neural network model

In [9]:
input_size  = 4
hidden_size = 6
output_size = 3
model = NeuralNetwork(
    input_size=input_size,
    hidden_size=hidden_size,
    output_size=output_size
)

Define loss functions for training the neural network

In [10]:
criterion = torch.nn.CrossEntropyLoss()
criterion

CrossEntropyLoss()

Define the optimizer for training the neural network

In [11]:
learning_rate = 0.0001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.0001
    maximize: False
    weight_decay: 0
)

Define function for training the model

In [12]:
ModelT      = Type[torch.nn.Module]
CriterionT  = Type[torch.nn.modules.loss._Loss]
OptimizerT  = Type[torch.optim.Optimizer]
DataLoaderT = Type[torch.utils.data.DataLoader]

def train(model: ModelT, criterion: CriterionT, optimizer: OptimizerT, dataloader: DataLoaderT, num_epochs: int, log_freq: int, log_verbose: bool = False) -> list[float]:
    loss_log = []
    for epoch in range(num_epochs):
        # Initialize loss accumulator
        loss_acc = 0.0
        # Train model
        for x_batch, y_batch in dataloader:
            # Forward pass
            y_pred = model(x_batch)
            y_loss = criterion(y_pred, y_batch)

            # Backward pass and optimization
            optimizer.zero_grad()
            y_loss.backward()
            optimizer.step()

            # Store loss
            loss_acc += y_loss
        
        # Log loss
        if (epoch + 1) % log_freq == 0:
            loss_log.append(loss_acc)
            if log_verbose:
                print(f"Epoch [{epoch+1}/{num_epochs}] | Loss: {loss_acc}")

    return loss_log


Train model

In [13]:
loss_log = train(
    model=model,
    criterion=criterion,
    optimizer=optimizer,
    dataloader=dataloader_train,
    num_epochs=1000,
    log_freq=100,
    log_verbose=True
)

Epoch [100/1000] | Loss: 19.465103149414062
Epoch [200/1000] | Loss: 14.550691604614258
Epoch [300/1000] | Loss: 11.619840621948242
Epoch [400/1000] | Loss: 10.129158020019531
Epoch [500/1000] | Loss: 8.988484382629395
Epoch [600/1000] | Loss: 7.906054973602295
Epoch [700/1000] | Loss: 6.851046085357666
Epoch [800/1000] | Loss: 5.927214622497559
Epoch [900/1000] | Loss: 5.141199588775635
Epoch [1000/1000] | Loss: 4.499476909637451


Define function for testing the model

In [14]:
def test(model: ModelT, dataloader: DataLoaderT) -> float:
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for x_batch, y_batch in dataloader:
            # Evaluate the model
            y_pred = model(x_batch)
            # Get indices of predicted classes
            _, predicted = torch.max(y_pred, -1)
            # Calculate number of predictions
            correct += (predicted == y_batch).sum().item()
            # Update accumulator for number of evaluations
            total += y_batch.size(0)

    # Calculate accuracy
    accuracy = correct / total

    return accuracy

Test the trained model

In [15]:
print(f"Model accuracy on test set: {test(model, dataloader_test)}")

Model accuracy on test set: 0.9666666666666667
